# Импорт

In [1]:
import sys
sys.path.append('../')

import momepy
import folium
import pyproj
import shapely
import osmnx as ox
import pandas as pd
import networkx as nx
import graphbuilder_v2
import geopandas as gpd

from pprint import pprint
from pyproj import Transformer

# Примеры работы

## Загрузка графа

In [ ]:
# В качестве начальных данных подается gpd.GeoDataFrame с Polygon, по которому нужно скачать граф
lo_polygon = ox.geocode_to_gdf('R176095', by_osmid=True).to_crs(epsg=32636)
spb_polygon = ox.geocode_to_gdf('R337422', by_osmid=True).to_crs(epsg=32636).buffer(3000)
city = lo_polygon.union(spb_polygon).to_crs(epsg=4326)
# city = ox.geocode_to_gdf('R1281563', by_osmid=True) # тестовый мини-город, чтобы долго не грузить

# city = ox.geocode_to_gdf('R1572051', by_osmid=True)

graph = graphbuilder_v2.get_graph_from_polygon(city, crs=32636, retain_all=False)

In [2]:
# Загрузка локального графа
graph = nx.read_graphml('data/graphml/graph.graphml')

In [5]:
# Перевод edges:list -> str
graph = graphbuilder_v2.convert_list_attr_to_str(graph)



In [6]:
nodes, edges = momepy.nx_to_gdf(graph, points=True, lines=True, spatial_weights=False)

In [ ]:
nx.write_graphml(graph, 'data/graphml/graph.graphml')

## Создание карты по каркасу

In [ ]:
import folium
from pyproj import Transformer

# Инициализация трансформера для преобразования из EPSG:32636 в EPSG:4326
transformer = Transformer.from_crs("epsg:32636", "epsg:4326")

# Выделение дорог с федеральным/региональным статусом
e = [(u, v, k) for u, v, k, d in graph.edges(data=True, keys=True) if d.get('reg') in([1, 2])]

m = folium.Map(tiles='CartoDB Dark_Matter')

# Создание FeatureGroup для слоев
lines_layer = folium.FeatureGroup(name='Lines', show=False)
nodes_layer = folium.FeatureGroup(name='Nodes', show=False)
exit_nodes_layer = folium.FeatureGroup(name='Exit Nodes', show=False)

# Преобразование и отображение узлов и линий
for u, v, k in e:
    start_y, start_x = graph.nodes[u]['y'], graph.nodes[u]['x']
    end_y, end_x = graph.nodes[v]['y'], graph.nodes[v]['x']
    
    # Преобразование координат
    start_lat, start_lon = transformer.transform(start_x, start_y)
    end_lat, end_lon = transformer.transform(end_x, end_y)
    
    # Создание линии
    line_coords = [
        [start_lat, start_lon],
        [end_lat, end_lon]
    ]
    folium.PolyLine(locations=line_coords, color='#AAFF01').add_to(lines_layer)
    
    # Условие для цвета CircleMarker и создание tooltip
    start_color = '#FE4F19' if graph.nodes[u].get('exit') == 1 else '#13D1FF'
    end_color =   '#FE4F19' if graph.nodes[v].get('exit') == 1 else '#13D1FF'
    
    start_tooltip = f"Exit: {graph.nodes[u].get('exit')}, Reg1: {graph.nodes[u].get('reg_1')}, Reg2: {graph.nodes[u].get('reg_2')}"
    end_tooltip =   f"Exit: {graph.nodes[v].get('exit')}, Reg1: {graph.nodes[v].get('reg_1')}, Reg2: {graph.nodes[v].get('reg_2')}"
    
    # Добавление CircleMarker для начального узла
    folium.CircleMarker(
        location=[start_lat, start_lon],
        radius=2,
        color=start_color,
        fill=True,
        opacity=1,
        fill_color=start_color,
        tooltip=start_tooltip
    ).add_to(exit_nodes_layer if graph.nodes[u].get('exit') == 1 else nodes_layer)
    
    # Добавление CircleMarker для конечного узла
    folium.CircleMarker(
        location=[end_lat, end_lon],
        radius=2,
        color=end_color,
        fill=True,
        opacity=1,
        fill_color=end_color,
        tooltip=end_tooltip
    ).add_to(exit_nodes_layer if graph.nodes[v].get('exit') == 1 else nodes_layer)

# Добавление слоев на карту
lines_layer.add_to(m)
nodes_layer.add_to(m)
exit_nodes_layer.add_to(m)

# Отображение границы полигона (например, города)
city.boundary.explore(m=m, color='#AA00FF', name='Cities', opacity=0.8)

# Добавление контроля слоев
folium.LayerControl().add_to(m)

# Сохранение карты
# Путь проверьте!!!
m.save('../data/html/rostov_graphbuilder2.html')

## Перевод graph -> GeoDataFrame

### Конвертация str -> geometry

In [ ]:
# Конвертация str -> geometry
graph = graphbuilder_v2.convert_geometry_from_wkt(graph)

In [ ]:
# Преобразуем граф в GeoDataFrames
nodes, edges = momepy.nx_to_gdf(graph, points=True, lines=True, spatial_weights=False)

## Вывод на карту определенный тип reg_status дороги

In [ ]:
# Разница между предыдущим и этим методом:
# Вариант выше рисовал узлы и ребра графа. Этот вариант рисует геометрию

edges = edges.to_crs(epsg=4326)
edges_filtered = edges[edges['reg'].isin([1, 2])]

m = folium.Map()

# Определение цветов для разных значений reg
color_map = {1: 'red', 2: 'blue', 3: 'green'}

# Добавление рёбер на карту
for _, row in edges_filtered.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=lambda feature, color=color_map[row['reg']]: {'color': color},
        tooltip=f"highway: {row['highway']}, ref: {row['ref']}, reg: {row['reg']}"
    ).add_to(m)

# Отображение карты
m.save('../data/html/roads.html')

## Проецирование точек на узлы графа

In [ ]:
# Пример проецирования точек на граф. Важно: пример для Ленинградской области и Санкт-Петербурга

lomonosov = ox.geocode_to_gdf('N411691832', by_osmid=True).to_crs(epsg=32636)[['geometry','name']]
lomonosov.loc[0, 'name'] = 'Ломоносов'

city_points = gpd.read_file('../data/geojsons/СНП ЛО.geojson')
rr_points = gpd.read_file('../data/geojsons/ЖД остановки.geojson').to_crs(epsg=32636)

cities = ["приозерск", "кировск", "кингисепп", "луга", 
          "лодейное поле", "гатчина", "тихвин", "тосно", 
          "выборг", "бокситогорск", "всеволожск", "волосово", 
          "волхов", "сосновый бор", "сланцы", "подпорожье", "кириши"]

# Выделение административных центров
city_points = city_points[
    (city_points['name'].str.lower().isin(cities)) &
    (city_points['rural settlement'].str.contains('административный центр'))
].sort_values('name')

city_points = city_points.to_crs(epsg=32636)
city_points = pd.concat([city_points, lomonosov]).reset_index(drop=True)

nodes_fil = nodes[(nodes['reg_1'] == True) | (nodes['reg_2'] == True)]
graph = graphbuilder_v2.assign_city_names_to_nodes(city_points, nodes_fil, graph, name_attr='city_name', node_id_attr='nodeID', name_col='name', max_distance=1200)
graph = graphbuilder_v2.assign_city_names_to_nodes(rr_points, nodes_fil, graph, name_attr='rr_name', node_id_attr='nodeID', name_col='NAME', max_distance=1200)

In [ ]:
import folium
from pyproj import Transformer

# Инициализация трансформера для преобразования из EPSG:32636 в EPSG:4326
transformer = Transformer.from_crs("epsg:32636", "epsg:4326")

# Инициализация карты
m = folium.Map(tiles='CartoDB Dark_Matter')

# Создание FeatureGroup для слоев
nodes_city_layer = folium.FeatureGroup(name='Nodes with City Names', show=False)
nodes_rr_layer = folium.FeatureGroup(name='Nodes with RR Names', show=False)
nodes_default_layer = folium.FeatureGroup(name='Other Nodes', show=False)
lines_layer_reg1 = folium.FeatureGroup(name='Lines reg 1', show=False)
lines_layer_reg2 = folium.FeatureGroup(name='Lines reg 2', show=False)
exit_nodes_layer = folium.FeatureGroup(name='Exit Nodes', show=False)

# Преобразование и отображение узлов и линий
for u, v, k, d in graph.edges(data=True, keys=True):
    start_y, start_x = graph.nodes[u]['y'], graph.nodes[u]['x']
    end_y, end_x = graph.nodes[v]['y'], graph.nodes[v]['x']
    
    # Преобразование координат
    start_lat, start_lon = transformer.transform(start_x, start_y)
    end_lat, end_lon = transformer.transform(end_x, end_y)
    
    # Создание линии
    line_coords = [
        [start_lat, start_lon],
        [end_lat, end_lon]
    ]
    
    if d.get('reg') == 1:
        folium.PolyLine(locations=line_coords, color='blue').add_to(lines_layer_reg1)
    elif d.get('reg') == 2:
        folium.PolyLine(locations=line_coords, color='green').add_to(lines_layer_reg2)
    
    # Проверка и добавление узлов
    for node, lat, lon in [(u, start_lat, start_lon), (v, end_lat, end_lon)]:
        if graph.nodes[node].get('exit') == 1:
            folium.CircleMarker(
                location=[lat, lon],
                radius=5,
                color='#FE4F19',
                fill=True,
                fill_color='#FE4F19',
                tooltip=f"Exit: {graph.nodes[node].get('exit')}, Reg1: {graph.nodes[node].get('reg_1')}, Reg2: {graph.nodes[node].get('reg_2')}"
            ).add_to(exit_nodes_layer)
        elif graph.nodes[node].get('city_name'):
            folium.CircleMarker(
                location=[lat, lon],
                radius=5,
                color='red',
                fill=True,
                fill_color='red',
                tooltip=str(graph.nodes[node]['city_name'])
            ).add_to(nodes_city_layer)
        elif graph.nodes[node].get('rr_name'):
            folium.CircleMarker(
                location=[lat, lon],
                radius=5,
                color='yellow',
                fill=True,
                fill_color='yellow',
                tooltip=str(graph.nodes[node]['rr_name'])
            ).add_to(nodes_rr_layer)
        elif graph.nodes[node].get('reg_1') == True or graph.nodes[node].get('reg_2'):
            folium.CircleMarker(
                location=[lat, lon],
                radius=5,
                color='gray',
                fill=True,
                fill_color='gray'
            ).add_to(nodes_default_layer)

# Добавление слоев на карту
nodes_rr_layer.add_to(m)
lines_layer_reg1.add_to(m)
lines_layer_reg2.add_to(m)
exit_nodes_layer.add_to(m)
nodes_city_layer.add_to(m)
nodes_default_layer.add_to(m)

# Добавление контроля слоев
folium.LayerControl().add_to(m)

# Сохранение карты
m.save('../data/html/LO_all_data.html')

## Выделение подграфа (каркаса)

In [ ]:
e = [(u, v, k) for u, v, k, d in graph.edges(data=True, keys=True) if d.get('reg') in([1, 2])]
subgraph = graph.edge_subgraph(e).copy()

## Assessment of the state of the territory

In [ ]:
# Загрузка локального графа
graph = nx.read_graphml('data/graphml/graph.graphml')
# Перевод nodes:str -> int, geometry:str -> geometry
graph = nx.convert_node_labels_to_integers(graph)
graph = graphbuilder_v2.convert_geometry_from_wkt(graph)
# Перевод edges:str -> list
graph = graphbuilder_v2.convert_list_attr_from_str(graph)